# ANALYSER

In [7]:
import Modele
from Variables import *
import csv
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import time
import statistics

In [11]:
# Idée de test 1 : nbr de voiture total sur la route hors fin


def nbr_voiture(route, direction, traffic, duree_feu, duree_test):

    for i in range(len(route)):
        for j in range(len(route[i])):
            if route[i][j] != 0 and route[i][j][0] == "Feu":
                route[i][j][2] = duree_feu

    res = []

    for temps in range(duree_test):

        route, direction, traffic = Modele.mouvement(route, direction, traffic, temps)

        compt = 0

        for i in range(len(traffic)):
            for j in range(len(traffic[i])):
                if isinstance(traffic[i][j], list):
                    if route[i][j] != 0 and route[i][j][0] == "Route":
                        compt += traffic[i][j].count(1)
                    elif route[i][j] != 0 and route[i][j][0] != "Fin":
                        compt += traffic[i][j][0]

        res.append(compt)

    return res


def test_1(route, direction, traffic, duree_min, duree_max, nbr_test, nbr_ite):

    res = []
    
    for duree in tqdm(range(duree_min, duree_max)):
        for i in range(nbr_test):
            compt = []
            compt.append(sum(nbr_voiture(route, direction, traffic, duree, nbr_ite)))
        res.append(statistics.mean(compt))
        
    return res


r = test_1(route_etude, direction_etude, traffic_etude, 5, 60, 10, 1000)
index = sorted(range(len(r)), key=lambda i: r[i])

print(r)
print([i + 5 for i in index])

  0%|          | 0/55 [00:00<?, ?it/s]

100%|██████████| 55/55 [00:41<00:00,  1.32it/s]

[7097, 7459, 6887, 6745, 7238, 6940, 8182, 7883, 6366, 6888, 7056, 7751, 7408, 8557, 6889, 8345, 8171, 8176, 8280, 8395, 8329, 9652, 8638, 9390, 8851, 8907, 8927, 9794, 8686, 8340, 8340, 9467, 9355, 9090, 9085, 9572, 9296, 9869, 9477, 10680, 9814, 10768, 8823, 10004, 10898, 9999, 9774, 9767, 10742, 10972, 11807, 10701, 11793, 12050, 11654]
[13, 8, 7, 14, 19, 10, 15, 5, 9, 17, 6, 16, 12, 21, 22, 11, 23, 25, 34, 35, 20, 24, 18, 27, 33, 47, 29, 30, 31, 39, 38, 41, 37, 28, 36, 43, 40, 26, 52, 51, 32, 45, 42, 50, 48, 44, 56, 53, 46, 49, 54, 59, 57, 55, 58]
